CREATE FLAG PARAMETER

In [0]:
dbutils.widgets.text('Incremental_flag','0')
Incremental_flag = dbutils.widgets.get("Incremental_flag")
print(Incremental_flag)                     


# CREATING DIMENSIONAL MODEL


In [0]:
df_src = spark.sql('''
Select distinct(Branch_id),BranchName from parquet.`abfss://silver1@latestadls.dfs.core.windows.net/carsales`''')

df_src.display()





# Dim Model Sink INITIAL and INCREMENTAL
 Load

In [0]:

if  spark.catalog.tableExists('cars_catalog.gold.dim_branch'):
    df_sink= spark.sql('''select  Dim_Branch_key, Branch_id,BranchName from cars_catalog.gold.dim_branch ''')

else:
    df_sink= spark.sql('''select 1 as Dim_Branch_key, Branch_id,BranchName from parquet.`abfss://silver1@latestadls.dfs.core.windows.net/carsales` where 1=0 ''')
    display(df_sink)



# Filtering New and Old Records


In [0]:
df_filter = df_src.join(df_sink,df_src.Branch_id == df_sink.Branch_id,'left').select(df_src.Branch_id,df_src.BranchName,df_sink.Dim_Branch_key)
df_filter.display()


In [0]:
df_filter_new = df_filter.filter(df_filter.Dim_Branch_key.isNull()).select(df_filter.Branch_id,df_filter.BranchName)
df_filter_new.display()

df_filter_old = df_filter.filter(df_filter.Dim_Branch_key.isNotNull())
df_filter_old.display()


# Create Surrogate Key

**fetch the Max Surrogate key from existing table**

In [0]:
if Incremental_flag == '0':
    max_value = 1
else:
    max_value_df = spark.sql("select max(Dim_Branch_key) from cars_catalog.gold.dim_Branch")
    max_value = max_value_df.collect()[0][0]
    print(max_value)

**create surrogate key and add max surrogate key**


In [0]:
from pyspark.sql.functions import lit, monotonically_increasing_id
df_filter_new= df_filter_new.withColumn('Dim_Branch_key', lit(max_value + monotonically_increasing_id()))
                                        
df_filter_new.display()
#df_filter_new.write.mode('append')

# Create Final DF- df_filter_old+- df_filter_new

In [0]:
df_final = df_filter_old.union(df_filter_new)
display(df_final)

# SCD TYPE -1 (UPSERT)

In [0]:
from delta.tables import DeltaTable

In [0]:
#incremental run 
if spark.catalog.tableExists('cars_catalog.gold.dim_Branch'):
    delta_tbl=DeltaTable.forPath(spark, 'abfss://gold1@latestadls.dfs.core.windows.net/dim_Branch')
    delta_tbl.alias("trg").merge(df_final.alias("src"),"trg.Branch_id = src.Branch_id").whenMatchedUpdateAll().whenNotMatchedInsertAll().execute()
# initial run
else:
    df_final.write.format('delta').mode('overwrite').option("path","abfss://gold1@latestadls.dfs.core.windows.net/dim_Branch").saveAsTable('cars_catalog.gold.dim_Branch')

In [0]:
%sql
select * from cars_catalog.gold.dim_Branch